<a href="https://colab.research.google.com/github/roxyrong/w266_project/blob/main/GPT_J_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Experiment 1: Baseline GPT-J 6B parameter model

The goal of this notebook is to construct a prompt design which reliably returns SQL code to test the pretrained text-to-SQL performance of the [GPT-J 6B parameter transformer model](https://huggingface.co/EleutherAI/gpt-j-6b).

## Notebook & Environment Setup

In [1]:
# Install non-natively available libraries
%%capture

#!pip install transformers
!pip install sentencepiece
!pip install accelerate -U
!pip install datasets
!pip install nltk

# Install required libraries for 8-bit runtime
!pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet accelerate

In [9]:
# Imports
import os
import sys
from typing import Dict, List
import subprocess
import collections
import json
import random
import numpy as np
import pandas as pd
import nltk
import torch
import pprint
from google.colab import drive


# Import GPT-J for text generation + AutoTokenizer
from transformers import GPTJForCausalLM, AutoTokenizer

# for evaluation
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# Set up Google Drive for data storage

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Github/w266_project


project_path = '/content/drive/MyDrive/Github/w266_project'
sys.path.append(project_path)

/content/drive/MyDrive/Github/w266_project


In [5]:
%pwd

'/content/drive/MyDrive/Github/w266_project'

In [6]:
# setup third_party drive with spider dataset and utilities, only need to do once
# %mkdir third_party
# %cd third_party
# !git clone https://github.com/taoyds/spider.git
# !git clone https://github.com/HKUNLP/UnifiedSKG.git

## Spider Data Set Structure Exploration

In [7]:
# Load Spider datasets
with open('spider/train_spider.json', 'r') as f:
    train_spider = pd.read_json(f)
with open('spider/train_others.json', 'r') as f:
    others_spider = pd.read_json(f)
with open('spider/dev.json', 'r') as f:
    dev_spider = pd.read_json(f)

In [8]:
train_spider.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."


In [9]:
# show first 5 elements of training set
# for i in range(0, 4):
#   print(train_spider.iloc[i])
#   print(train_spider.iloc[i]['question'])
#   print(train_spider.iloc[i]['query'])

# Show 5 random elements of training set db_id, question, query
sample = train_spider.sample(n=5)
for i in range(0,5):
  print(sample.iloc[i]['db_id'])
  print(sample.iloc[i]['question'])
  print(sample.iloc[i]['query'])
  print('\n')

match_season
Who are the players from Indonesia?
SELECT T2.Player FROM country AS T1 JOIN match_season AS T2 ON T1.Country_id  =  T2.Country WHERE T1.Country_name  =  "Indonesia"


department_store
What is the id and name of the staff who has been assigned for the least amount of time?
SELECT T1.staff_id ,  T1.staff_name FROM staff AS T1 JOIN Staff_Department_Assignments AS T2 ON T1.staff_id  =  T2.staff_id ORDER BY date_assigned_to - date_assigned_from LIMIT 1


tracking_grants_for_research
List the project details of the project both producing patent and paper as outcomes.
SELECT T1.project_details FROM Projects AS T1 JOIN Project_outcomes AS T2 ON T1.project_id  =  T2.project_id WHERE T2.outcome_code  =  'Paper' INTERSECT SELECT T1.project_details FROM Projects AS T1 JOIN Project_outcomes AS T2 ON T1.project_id  =  T2.project_id WHERE T2.outcome_code  =  'Patent'


voter_2
Find the distinct last names of all the students who have president votes and whose advisor is not 2192.
SELECT

In [10]:
# Load schema for all tables
with open('spider/tables.json', 'r') as f:
    schema_df = pd.read_json(f)


In [11]:
# Helper Function to extract target schemas from Spider json to a dict
# for training and prompt usage.
def _get_schema_string(table_json):
    """Returns the schema serialized as a string."""
    table_id_to_column_names = collections.defaultdict(list)
    for table_id, name in table_json["column_names_original"]:
        table_id_to_column_names[table_id].append(name.lower())
        tables = table_json["table_names_original"]

    table_strings = []
    for table_id, table_name in enumerate(tables):
        column_names = table_id_to_column_names[table_id]
        table_string = " | %s : %s" % (table_name.lower(), " , ".join(column_names))
        table_strings.append(table_string)

    return "".join(table_strings)

schema_dict = {}
for idx, row in schema_df.iterrows():
    db_id = row['db_id']
    schema = _get_schema_string(row)
    schema_dict[db_id] = schema

In [12]:
print(schema_dict.keys())

dict_keys(['perpetrator', 'college_2', 'flight_company', 'icfp_1', 'body_builder', 'storm_record', 'pilot_record', 'race_track', 'academic', 'department_store', 'music_4', 'insurance_fnol', 'cinema', 'decoration_competition', 'phone_market', 'store_product', 'assets_maintenance', 'student_assessment', 'dog_kennels', 'music_1', 'company_employee', 'farm', 'solvency_ii', 'city_record', 'swimming', 'flight_2', 'election', 'manufactory_1', 'debate', 'network_2', 'local_govt_in_alabama', 'climbing', 'e_learning', 'scientist_1', 'ship_1', 'entertainment_awards', 'allergy_1', 'imdb', 'products_for_hire', 'candidate_poll', 'chinook_1', 'flight_4', 'pets_1', 'dorm_1', 'journal_committee', 'flight_1', 'medicine_enzyme_interaction', 'local_govt_and_lot', 'station_weather', 'shop_membership', 'driving_school', 'concert_singer', 'music_2', 'sports_competition', 'railway', 'inn_1', 'museum_visit', 'browser_web', 'baseball_1', 'architecture', 'csu_1', 'tracking_orders', 'insurance_policies', 'gas_com

In [13]:
print(schema_dict['cre_Doc_Control_Systems'], '\n')
print(schema_dict['shop_membership'], '\n')
print(schema_dict['bike_1'], '\n')
print(schema_dict['cre_Doc_Tracking_DB'], '\n')
print(schema_dict['twitter_1'], '\n')
print(schema_dict['journal_committee'], '\n')

 | ref_document_types : document_type_code , document_type_description | roles : role_code , role_description | addresses : address_id , address_details | ref_document_status : document_status_code , document_status_description | ref_shipping_agents : shipping_agent_code , shipping_agent_name , shipping_agent_description | documents : document_id , document_status_code , document_type_code , shipping_agent_code , receipt_date , receipt_number , other_details | employees : employee_id , role_code , employee_name , other_details | document_drafts : document_id , draft_number , draft_details | draft_copies : document_id , draft_number , copy_number | circulation_history : document_id , draft_number , copy_number , employee_id | documents_mailed : document_id , mailed_to_address_id , mailing_date 

 | member : member_id , card_number , name , hometown , level | branch : branch_id , name , open_year , address_road , city , membership_amount | membership_register_branch : member_id , branch_

A few examples from Spider to help with the prompt model:

Schema string:

 | department : department_id , name , creation , ranking , budget_in_billions , num_employees | head : head_id , name , born_state , age | management : department_id , head_id , temporary_acting

 Question:

 How many heads of the departments are older than 56 ?

 Query:

 SELECT count(*) FROM head WHERE age  >  56

 ____
flight_4

What is the name, city, and country of the airport with the lowest altitude?
SELECT name ,  city ,  country FROM airports ORDER BY elevation LIMIT 1


aircraft

what is the name and age of the youngest winning pilot?
SELECT t1.name ,  t1.age FROM pilot AS t1 JOIN MATCH AS t2 ON t1.pilot_id  =  t2.winning_pilot ORDER BY t1.age LIMIT 1


icfp_1

How many papers are "Atsushi Ohori" the author of?
SELECT count(*) FROM authors AS t1 JOIN authorship AS t2 ON t1.authid  =  t2.authid JOIN papers AS t3 ON t2.paperid  =  t3.paperid WHERE  t1.fname  =  "Atsushi" AND t1.lname  =  "Ohori"


activity_1

How many faculty members participate in each activity? Return the activity names and the number of faculty members.
SELECT T1.activity_name ,  count(*) FROM Activity AS T1 JOIN Faculty_participates_in AS T2 ON T1.actID  =  T2.actID GROUP BY T1.actID

journal_committee

Show the names of editors of age either 24 or 25.
SELECT Name FROM editor WHERE Age  =  24 OR Age  =  25

## Set up and load stock pretrained float 16 GPT-J model


In [14]:
# # Specify GPU for computations
# device = "cuda"

# # Load pretrained float16 GPT-J to GPU from Huggingface
# # model = GPTJForCausalLM.from_pretrained(
# #     "EleutherAI/gpt-j-6B",
# #     revision="float16",
# #     torch_dtype=torch.float16,
# # ).to(device)

# # Load pretrained float16 GPT-J from Google Drive
# model = GPTJForCausalLM.from_pretrained(
#     "pretrained_GPT-J",
# ).to(device)

In [15]:
#Save model to Google Drive to avoid long download times - only needed when needed :-)
# model.save_pretrained("pretrained_GPT-J", from_pt=True)

## Load tokenizer

In [16]:
# # Load appropriate tokenizer for model
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B", padding_side="left")


## Set up an 8-bit version

Memory runs out with stock version for batch inference, so fine tuning is going to be tough.

[Huggingface page for gpt-j-6B-8bit](https://huggingface.co/hivemind/gpt-j-6B-8bit)

[source notebook](https://colab.research.google.com/drive/1ft6wQU0BhqG5PRlwgaZJv2VukKKjU4Es#scrollTo=p0dy1ZFwClcq)

In [17]:
import transformers

import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd

from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise

from tqdm.auto import tqdm

In [18]:
class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias

    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output += self.adapter(input)
        return output

    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)

    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"


class DequantizeAndLinear(torch.autograd.Function):
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias)

    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias


class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None

    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output

    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)

    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"


def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)

    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)


def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr(
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

In [19]:
class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

In [20]:
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B", padding_side="left")

In [21]:
model = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): FrozenBNBLinear(4096, 50400)
)

## Experimentation to build a workable prompt

In [22]:


# Test prompt
prompt = ("\n-- Parse the question into SQL based on the given table below.--\n\n"
          " | routes : rid , dst_apid , dst_ap , src_apid , src_ap , alid , airline , codeshare | airports : apid , name , city , country , x , y , elevation , iata , icao | airlines : alid , name , iata , icao , callsign , country , active"
          "\n Based on this schema, create an ANSI-92 SQL Query to answer the following question:\n"
          "Q:What is the name, city, and country of the airport with the lowest altitude?"
          "\n Return the SQL query ONLY. Do not include any additional explanation."
          )
#prompt = ("# Question: What is the name, city, and country of the airport with the lowest altitude?"
          # "# Schema: | routes : rid , dst_apid , dst_ap , src_apid , src_ap , alid , airline , codeshare | airports : apid , name , city , country , x , y , elevation , iata , icao | airlines : alid , name , iata , icao , callsign , country , active"
          # "SQL Query:")

# Tokenize prompt, load to GPU with model
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Generate response tokens from GPT-J
gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.1,
    max_new_tokens=128,
    repetition_penalty=1.1,
    top_p=1,
)

# Retrieve and print response
gen_text = tokenizer.batch_decode(gen_tokens)[0]

print(gen_text)

# import re

# s = gen_text

# # Use re.findall to get all matches
# matches = re.search(r"(SELECT.*?;)", s, re.DOTALL)

# if match:
#     print("Query only: \n", match)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



-- Parse the question into SQL based on the given table below.--

 | routes : rid, dst_apid, dst_ap, src_apid, src_ap, alid, airline, codeshare | airports : apid, name, city, country, x, y, elevation, iata, icao | airlines : alid, name, iata, icao, callsign, country, active
 Based on this schema, create an ANSI-92 SQL Query to answer the following question:
Q:What is the name, city, and country of the airport with the lowest altitude?
 Return the SQL query ONLY. Do not include any additional explanation.

A:

This should work for you:
SELECT a.* FROM airports AS a
INNER JOIN (
  SELECT alid, MIN(elevation) AS minElevation
  FROM airports
  GROUP BY alid
) AS b ON a.alid = b.alid AND a.elevation = b.minElevation;

SQL Fiddle Demo

<|endoftext|>


In [23]:
# Load appropriate tokenizer for model
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B", padding_side="left")

# Test prompt
prompt = ("\n-- Parse the question into SQL based on the given table below.--\n\n"
          "| department : department_id , name , creation , ranking , budget_in_billions , num_employees | head : head_id , name , born_state , age | management : department_id , head_id , temporary_acting"
          "\n Based on this schema, create an ANSI-92 SQL Query to answer the following question:\n"
          "Q:How many heads of the departments are older than 56 ?"
          "\n As answer, return the SQL query ONLY. Do not include any additional explanation."
          )

# Tokenize prompt, load to GPU with model
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Generate response tokens from GPT-J
gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.1,
    max_new_tokens=128,
    repetition_penalty=1.1,
    top_p=1,
)

# Retrieve and print response
gen_text = tokenizer.batch_decode(gen_tokens)[0]

print(gen_text)

# import re

# s = gen_text

# # Use re.findall to extract the first query
# matches = re.search(r"(SELECT.*?;)", s, re.DOTALL)

# if match:
#     print("Query only: \n", match)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



-- Parse the question into SQL based on the given table below.--

| department : department_id, name, creation, ranking, budget_in_billions, num_employees | head : head_id, name, born_state, age | management : department_id, head_id, temporary_acting
 Based on this schema, create an ANSI-92 SQL Query to answer the following question:
Q:How many heads of the departments are older than 56?
 As answer, return the SQL query ONLY. Do not include any additional explanation.

A:

This is a simple example of how you can use the "with" clause in PostgreSQL.  The basic idea is that you want to select all rows from the table and then filter them using the where clause.  In your case, you want to select all rows from the table and then filter them by the column "age".  You can do this with the "where" clause.  Here's an example:
select * from employees
where age > 56;

<|endoftext|>


In [24]:
# Try few shot training
# Load appropriate tokenizer for model (not good to repeat here?)
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B", padding_side="left")

# cre_Doc_Control_Systems
# Which employee has showed up in most circulation history documents. List the employee's name and the number of drafts and copies.
# SELECT Employees.employee_name , count(*) FROM Employees JOIN Circulation_History ON Circulation_History.employee_id = Employees.employee_id GROUP BY Circulation_History.document_id , Circulation_History.draft_number , Circulation_History.copy_number ORDER BY count(*) DESC LIMIT 1;
# | ref_document_types : document_type_code , document_type_description | roles : role_code , role_description | addresses : address_id , address_details | ref_document_status : document_status_code , document_status_description | ref_shipping_agents : shipping_agent_code , shipping_agent_name , shipping_agent_description | documents : document_id , document_status_code , document_type_code , shipping_agent_code , receipt_date , receipt_number , other_details | employees : employee_id , role_code , employee_name , other_details | document_drafts : document_id , draft_number , draft_details | draft_copies : document_id , draft_number , copy_number | circulation_history : document_id , draft_number , copy_number , employee_id | documents_mailed : document_id , mailed_to_address_id , mailing_date


# shop_membership
# What are names for top three branches with most number of membership?
# SELECT name FROM branch ORDER BY membership_amount DESC LIMIT 3
# | member : member_id , card_number , name , hometown , level | branch : branch_id , name , open_year , address_road , city , membership_amount | membership_register_branch : member_id , branch_id , register_year | purchase : member_id , branch_id , year , total_pounds


# bike_1
# What is the id of the trip that has the shortest duration?
# SELECT id FROM trip ORDER BY duration LIMIT 1
# | station : id , name , lat , long , dock_count , city , installation_date | status : station_id , bikes_available , docks_available , time | trip : id , duration , start_date , start_station_name , start_station_id , end_date , end_station_name , end_station_id , bike_id , subscription_type , zip_code | weather : date , max_temperature_f , mean_temperature_f , min_temperature_f , max_dew_point_f , mean_dew_point_f , min_dew_point_f , max_humidity , mean_humidity , min_humidity , max_sea_level_pressure_inches , mean_sea_level_pressure_inches , min_sea_level_pressure_inches , max_visibility_miles , mean_visibility_miles , min_visibility_miles , max_wind_speed_mph , mean_wind_speed_mph , max_gust_speed_mph , precipitation_inches , cloud_cover , events , wind_dir_degrees , zip_code


# cre_Doc_Tracking_DB
# What are all the document type codes and document type names?
# SELECT document_type_code ,  document_type_name FROM Ref_document_types
# | ref_document_types : document_type_code , document_type_name , document_type_description | ref_calendar : calendar_date , day_number | ref_locations : location_code , location_name , location_description | roles : role_code , role_name , role_description | all_documents : document_id , date_stored , document_type_code , document_name , document_description , other_details | employees : employee_id , role_code , employee_name , gender_mfu , date_of_birth , other_details | document_locations : document_id , location_code , date_in_location_from , date_in_locaton_to | documents_to_be_destroyed : document_id , destruction_authorised_by_employee_id , destroyed_by_employee_id , planned_destruction_date , actual_destruction_date , other_details


# twitter_1
# Find the average number of followers for the users who do not have any tweet.
# SELECT avg(followers) FROM user_profiles WHERE UID NOT IN (SELECT UID FROM tweets)
# )
# | follows : f1 , f2 | tweets : id , uid , text , createdate | user_profiles : uid , name , email , partitionid , followers

          #  ###
          #  Context: | station : id , name , lat , long , dock_count , city , installation_date | status : station_id , bikes_available , docks_available , time | trip : id , duration , start_date , start_station_name , start_station_id , end_date , end_station_name , end_station_id , bike_id , subscription_type , zip_code | weather : date , max_temperature_f , mean_temperature_f , min_temperature_f , max_dew_point_f , mean_dew_point_f , min_dew_point_f , max_humidity , mean_humidity , min_humidity , max_sea_level_pressure_inches , mean_sea_level_pressure_inches , min_sea_level_pressure_inches , max_visibility_miles , mean_visibility_miles , min_visibility_miles , max_wind_speed_mph , mean_wind_speed_mph , max_gust_speed_mph , precipitation_inches , cloud_cover , events , wind_dir_degrees , zip_code
          #  Question:What is the id of the trip that has the shortest duration?
          #  Answer: SELECT id FROM trip ORDER BY duration LIMIT 1
          #  ###
          #  Context: | follows : f1 , f2 | tweets : id , uid , text , createdate | user_profiles : uid , name , email , partitionid , followers
          #  Question: What are all the document type codes and document type names?
          #  Answer: SELECT document_type_code ,  document_type_name FROM Ref_document_types


prompt = ("""Context: | ref_document_types : document_type_code , document_type_description | roles : role_code , role_description | addresses : address_id , address_details | ref_document_status : document_status_code , document_status_description | ref_shipping_agents : shipping_agent_code , shipping_agent_name , shipping_agent_description | documents : document_id , document_status_code , document_type_code , shipping_agent_code , receipt_date , receipt_number , other_details | employees : employee_id , role_code , employee_name , other_details | document_drafts : document_id , draft_number , draft_details | draft_copies : document_id , draft_number , copy_number | circulation_history : document_id , draft_number , copy_number , employee_id | documents_mailed : document_id , mailed_to_address_id , mailing_date
           Question: Which employee has showed up in most circulation history documents. List the employee's name and the number of drafts and copies.\n"
           Answer: SELECT Employees.employee_name , count(*) FROM Employees JOIN Circulation_History ON Circulation_History.employee_id = Employees.employee_id GROUP BY Circulation_History.document_id , Circulation_History.draft_number , Circulation_History.copy_number ORDER BY count(*) DESC LIMIT 1;
           ###
           Context:| member : member_id , card_number , name , hometown , level | branch : branch_id , name , open_year , address_road , city , membership_amount | membership_register_branch : member_id , branch_id , register_year | purchase : member_id , branch_id , year , total_pounds
           Question: What are names for top three branches with most number of membership?
           Answer: SELECT name FROM branch ORDER BY membership_amount DESC LIMIT 3
           ###
           Context: | department : department_id , name , creation , ranking , budget_in_billions , num_employees | head : head_id , name , born_state , age | management : department_id , head_id , temporary_acting
           Question: How many heads of the departments are older than 56?
           Answer: """

)

# prompt = ("""[Text]: Helena Smith founded Core.ai 2 years ago. She is now the CEO and CTO of the company and is building a team of highly skilled developers in machine learning and natural language processing.
#         [Position]: CEO and CTO
#         ###
#         [Text]: Tech Robotics is a robot automation company specialized in AI driven robotization. Its Chief Technology Officer, Max Smith, says a new wave of improvements should be expected for next year.
#         [Position]: Chief Technology Officer
#         ###
#         [Text]: François is a Go developer. He mostly works as a freelancer but is open to any kind of job offering!
#         [Position]: Go developer
#         ###
#         [Text]: Maxime is a data scientist at Auto Dataset, and he's been working there for 1 year.
#         [Position]:"""
# )

# Tokenize prompt, load to GPU with model
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Generate response tokens from GPT-J
gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    num_beams=1,
    temperature=0.1,
    max_new_tokens=128,
    repetition_penalty=1.1,
    top_p=1,
    early_stopping=True
)

# Retrieve generated text
gen_text = tokenizer.batch_decode(gen_tokens)[0]

# Remove prompt from generated text and print
# Split by 'Answer' and take the fourth part
part_after_third_answer = gen_text.split('Answer:')[3]

# Split this part by '###' and take the first part
extracted_string = part_after_third_answer.split('###')[0]

# Remove leading and trailing whitespaces
extracted_string = extracted_string.strip()

print(extracted_string)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


SELECT COUNT(*) FROM Department WHERE age > 56


In [30]:
#two cells to set up file structure
# Save model output to Drive
model_name = "GPT-J"
technique = "pretrained"
version = 1
# checkpoint = "checkpoint-3504"
inference_model = "GPT-J-standard"

folder_name = f"{model_name}_{technique}_{version}"
# train_path = f"results/{folder_name}"
# model_path = train_path + f'/{folder_name}'
# last_check_point = train_path + f'/checkpoint-2190'
result_path = f'results/{folder_name}/predicted_result_{inference_model}_{version}.txt'


# print("train_path:", train_path)
# print("model_path:", model_path)
print("result_path:", result_path)

# with open(result_path, 'w') as f:
#     for idx, output in enumerate(outputs):
#         db_id = dev_spider.iloc[idx]['db_id']
#         f.write(output + '\t' + db_id + '\n')

result_path: results/GPT-J_pretrained_1/predicted_result_GPT-J-standard_1.txt


In [32]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!touch results/GPT-J_pretrained_1/predicted_result_GPT-J-standard_1.txt

In [25]:
print(prompt)

Context: | ref_document_types : document_type_code , document_type_description | roles : role_code , role_description | addresses : address_id , address_details | ref_document_status : document_status_code , document_status_description | ref_shipping_agents : shipping_agent_code , shipping_agent_name , shipping_agent_description | documents : document_id , document_status_code , document_type_code , shipping_agent_code , receipt_date , receipt_number , other_details | employees : employee_id , role_code , employee_name , other_details | document_drafts : document_id , draft_number , draft_details | draft_copies : document_id , draft_number , copy_number | circulation_history : document_id , draft_number , copy_number , employee_id | documents_mailed : document_id , mailed_to_address_id , mailing_date
           Question: Which employee has showed up in most circulation history documents. List the employee's name and the number of drafts and copies.
"
           Answer: SELECT Employees

In [26]:
print(gen_text)

Context: | ref_document_types : document_type_code, document_type_description | roles : role_code, role_description | addresses : address_id, address_details | ref_document_status : document_status_code, document_status_description | ref_shipping_agents : shipping_agent_code, shipping_agent_name, shipping_agent_description | documents : document_id, document_status_code, document_type_code, shipping_agent_code, receipt_date, receipt_number, other_details | employees : employee_id, role_code, employee_name, other_details | document_drafts : document_id, draft_number, draft_details | draft_copies : document_id, draft_number, copy_number | circulation_history : document_id, draft_number, copy_number, employee_id | documents_mailed : document_id, mailed_to_address_id, mailing_date
           Question: Which employee has showed up in most circulation history documents. List the employee's name and the number of drafts and copies.
"
           Answer: SELECT Employees.employee_name, count(*)

# Construct prompts for baseline test

## Baseline test of pretrained GPT-J for Spider dataset

In [27]:
# Instruction style prompt
# prefix = '\n-- Parse the question into SQL based on the given table below.--\n\n'
# infix = '\n Based on this schema, create an ANSI-92 SQL Query to answer the following question:\n'
# postfix = '\n Your response should be in SQL.'


# train_spider['schema'] = train_spider['db_id'].map(schema_dict)
# train_spider['prompt'] = prefix + train_spider['schema'] + infix + train_spider['question'] + postfix

# others_spider['schema'] = others_spider['db_id'].map(schema_dict)
# others_spider['prompt'] = prefix + others_spider['schema'] + infix + others_spider['question']

# dev_spider['schema'] = dev_spider['db_id'].map(schema_dict)
# dev_spider['prompt'] = prefix + dev_spider['schema'] + infix + dev_spider['question'] + postfix

In [28]:
# Few-shot training style prompt
fixed_few_shot_prefix = """Context: | ref_document_types : document_type_code , document_type_description | roles : role_code , role_description | addresses : address_id , address_details | ref_document_status : document_status_code , document_status_description | ref_shipping_agents : shipping_agent_code , shipping_agent_name , shipping_agent_description | documents : document_id , document_status_code , document_type_code , shipping_agent_code , receipt_date , receipt_number , other_details | employees : employee_id , role_code , employee_name , other_details | document_drafts : document_id , draft_number , draft_details | draft_copies : document_id , draft_number , copy_number | circulation_history : document_id , draft_number , copy_number , employee_id | documents_mailed : document_id , mailed_to_address_id , mailing_date
           Question: Which employee has showed up in most circulation history documents. List the employee's name and the number of drafts and copies.\n"
           Answer: SELECT Employees.employee_name , count(*) FROM Employees JOIN Circulation_History ON Circulation_History.employee_id = Employees.employee_id GROUP BY Circulation_History.document_id , Circulation_History.draft_number , Circulation_History.copy_number ORDER BY count(*) DESC LIMIT 1;
           ###
           Context:| member : member_id , card_number , name , hometown , level | branch : branch_id , name , open_year , address_road , city , membership_amount | membership_register_branch : member_id , branch_id , register_year | purchase : member_id , branch_id , year , total_pounds
           Question: What are names for top three branches with most number of membership?
           Answer: SELECT name FROM branch ORDER BY membership_amount DESC LIMIT 3
           ###
           Context: """

fixed_few_shot_infix = "\n Question: "
fixed_few_shot_postfix = "\n Answer: "

train_spider['schema'] = train_spider['db_id'].map(schema_dict)
train_spider['prompt'] = fixed_few_shot_prefix + train_spider['schema'] + fixed_few_shot_infix + train_spider['question'] + fixed_few_shot_postfix

others_spider['schema'] = others_spider['db_id'].map(schema_dict)
others_spider['prompt'] = fixed_few_shot_prefix + others_spider['schema'] + fixed_few_shot_infix + others_spider['question'] + fixed_few_shot_postfix

dev_spider['schema'] = dev_spider['db_id'].map(schema_dict)
dev_spider['prompt'] = fixed_few_shot_prefix + dev_spider['schema'] + fixed_few_shot_infix + dev_spider['question'] + fixed_few_shot_postfix

In [29]:
dev_spider['prompt'][0]

'Context: | ref_document_types : document_type_code , document_type_description | roles : role_code , role_description | addresses : address_id , address_details | ref_document_status : document_status_code , document_status_description | ref_shipping_agents : shipping_agent_code , shipping_agent_name , shipping_agent_description | documents : document_id , document_status_code , document_type_code , shipping_agent_code , receipt_date , receipt_number , other_details | employees : employee_id , role_code , employee_name , other_details | document_drafts : document_id , draft_number , draft_details | draft_copies : document_id , draft_number , copy_number | circulation_history : document_id , draft_number , copy_number , employee_id | documents_mailed : document_id , mailed_to_address_id , mailing_date\n           Question: Which employee has showed up in most circulation history documents. List the employee\'s name and the number of drafts and copies.\n"\n           Answer: SELECT Empl

## Run baseline test prior to fine tuning


In [33]:
torch.cuda.empty_cache()

In [34]:
# evaluate
torch.cuda.empty_cache()

max_length = 128

tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

# Define a list to store outputs
outputs = []

# Process each prompt individually
for prompt in dev_spider['prompt']:
    #print("Prompt ", i)

    input_ids = tokenizer(
            prompt,
            return_tensors='pt',
            truncation=True,
            padding=True,
        ).input_ids.to(device)

    gen_tokens = model.generate(
                input_ids,
                do_sample=True,
                num_beams=1,
                temperature=0.1,
                max_new_tokens=128,
                repetition_penalty=1.1,
                top_p=1,
                early_stopping=True
        )

    #print("Output: ", tokenizer.decode(gen_tokens[0], skip_special_tokens=True))

    # Retrieve generated text
    gen_text = tokenizer.batch_decode(gen_tokens)[0]

    # Remove prompt from generated text and print
    # Split by 'Answer' and take the fourth part
    part_after_third_answer = gen_text.split('Answer:')[3]

    # Split this part by '###' and take the first part
    extracted_string = part_after_third_answer.split('###')[0]

    # Remove leading and trailing whitespaces
    extracted_string = extracted_string.strip()
    #print(extracted_string)
    outputs.append(extracted_string)

    # Clean GPU cache after each iteration to free up memory
    torch.cuda.empty_cache()



# Retrieve responses
#gen_text = tokenizer.batch_decode(gen_tokens)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [37]:
len(outputs)

1034

In [38]:
# Save model output to Drive
model_name = "GPT-J"
technique = "pretrained"
version = 1
# checkpoint = "checkpoint-3504"
inference_model = "GPT-J-standard"

folder_name = f"{model_name}_{technique}_{version}"
# train_path = f"results/{folder_name}"
# model_path = train_path + f'/{folder_name}'
# last_check_point = train_path + f'/checkpoint-2190'
result_path = f'results/{folder_name}/predicted_result_{inference_model}_{version}.txt'


# print("train_path:", train_path)
# print("model_path:", model_path)
print("result_path:", result_path)

with open(result_path, 'w', encoding='utf-8') as f:
    for idx, output in enumerate(outputs):
        db_id = dev_spider.iloc[idx]['db_id']
        f.write(output + '\t' + db_id + '\n')

result_path: results/GPT-J_pretrained_1/predicted_result_GPT-J-standard_1.txt


In [10]:
# evaluate results
eval_path = f"third_party/spider/evaluation.py"
gold = f"third_party/spider/evaluation_examples/gold_example.txt"
pred = f"results/GPT-J_pretrained_1/predicted_result_GPT-J-standard_1_clean.txt"
db_dir = f"spider/database"
table = f"spider/tables.json"
etype = "all"

cmd_str = f"python3 \"{eval_path}\" --gold \"{gold}\" --pred \"{pred}\" --db \"{db_dir}\" --table \"{table}\" --etype {etype} "
result = subprocess.run(cmd_str, shell=True, capture_output=True, text=True)

In [12]:
# print results
pp = pprint.PrettyPrinter(width=160)
#pp.pprint(result.stdout[-4633:])
pp.pprint(result.stdout)


('eval_err_num:1\n'
 'easy pred: SELECT COUNT(*) FROM Singers WHERE singer_id IN (SELECT DISTINCT singer_id FROM Singer_In_Concert)\n'
 'easy gold: SELECT count(*) FROM singer\n'
 '\n'
 'eval_err_num:2\n'
 'easy pred: SELECT COUNT(*) FROM Singers\n'
 'easy gold: SELECT count(*) FROM singer\n'
 '\n'
 'medium pred: SELECT name, country, age FROM Singer WHERE age > (SELECT MAX(age) FROM Singer) ORDER BY age ASC LIMIT 10\n'
 'medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC\n'
 '\n'
 'eval_err_num:3\n'
 'medium pred: SELECT name, country, age FROM singers WHERE is_male = 0 ORDER BY age DESC ;\n'
 'medium gold: SELECT name ,  country ,  age FROM singer ORDER BY age DESC\n'
 '\n'
 'eval_err_num:4\n'
 "medium pred: SELECT AVG(age) AS avg_age, MIN(age) AS min_age, MAX(age) AS max_age FROM singer WHERE country = 'FR' AND is_male = 0 GROUP BY singer_id\n"
 "medium gold: SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'\n"
 '\n'
 'eval_err_num:

In [13]:
pprint.pprint(result.stderr)

('Traceback (most recent call last):\n'
 '  File '
 '"/content/drive/MyDrive/Github/w266_project/third_party/spider/evaluation.py", '
 'line 868, in <module>\n'
 '    evaluate(gold, pred, db_dir, etype, kmaps)\n'
 '  File '
 '"/content/drive/MyDrive/Github/w266_project/third_party/spider/evaluation.py", '
 'line 547, in evaluate\n'
 '    exec_score = eval_exec_match(db, p_str, g_str, p_sql, g_sql)\n'
 '  File '
 '"/content/drive/MyDrive/Github/w266_project/third_party/spider/evaluation.py", '
 'line 628, in eval_exec_match\n'
 '    q_res = cursor.fetchall()\n'
 "sqlite3.OperationalError: Could not decode to UTF-8 column 'last_name' with "
 "text 'Treyes Albarrac��N'\n")


In [49]:
!pwd

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
pwd: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
